用tensorflow实现有两个隐藏层的神经网络，无激活函数，损失函数用softmax-cross-entrpy

## MNIST 数据库概览

MNIST手写数字数据集. 共有60,000张训练图片，10,000张测试图片. 所有图片已做过归一化与中心化，尺寸统一为28x28，像素值范围从0到1. 为了简单期间，输入时每张图片不以28x28的矩阵形式输入，而是按列排列，转为一维的长度为784=28x28的特征，输入神经网络。

![MNIST 数据集](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

In [2]:
import tensorflow as tf

#import mnist data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data', one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [17]:
print(mnist.train.next_batch(10)[1])

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [24]:
#超参数
batch_size = 500
learning_rate = 0.1
num_steps = 500
display_step = 100

n_hidden_1 = 256
n_hidden_2 = 256
num_input = 784
num_classes = 10

#初始化权重和偏置
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}

biaes = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}


In [11]:
#定义网络结构
def neural_net(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biaes['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biaes['b2'])
    out = tf.add(tf.matmul(layer_2, weights['out']), biaes['out'])
    return out

In [22]:
X = tf.placeholder("float", [None, 784])
Y = tf.placeholder("float", [None, 10])

logits = neural_net(X)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

#计算准确率
correct_pred = tf.equal(tf.argmax(logits, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init  = tf.global_variables_initializer()

In [27]:
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(train_op, feed_dict={X:batch_x,
                                      Y:batch_y})
        if step % display_step == 0 or step == 1:
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X:batch_x, Y:batch_y})
            print("step: {}, minibatch_loss:{:.4f}, train_accuracy:{:.4f}".format(step, loss, acc))
        
    print("train finished")
        
    test_accuracy = sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels})
    print("test accuracy ", test_accuracy)

step: 1, minibatch_loss:11368.1943, train_accuracy:0.4040
step: 100, minibatch_loss:218.4454, train_accuracy:0.8860
step: 200, minibatch_loss:77.6302, train_accuracy:0.8840
step: 300, minibatch_loss:64.3572, train_accuracy:0.8760
step: 400, minibatch_loss:34.1958, train_accuracy:0.8960
step: 500, minibatch_loss:24.9768, train_accuracy:0.9000
train finished
test accuracy  0.8833


In [29]:
    print(mnist.test.images.shape)

(10000, 784)
